In [1]:
import pandas as pd

In [4]:
import psutil

# Get memory information
memory_info = psutil.virtual_memory()

# Total physical memory (RAM)
total_memory = memory_info.total

# Available physical memory (RAM)
available_memory = memory_info.available

# Print the results
print(f"Total Memory: {total_memory / (1024 ** 3):.2f} GB")
print(f"Available Memory: {available_memory / (1024 ** 3):.2f} GB")


Total Memory: 7.89 GB
Available Memory: 1.70 GB


# import pandas as pd

# Specify the path to your large JSON file
file_path = '../datasets/techmap-jobs_us_2023-05-05.json'

# Specify the desired chunk size (number of rows per chunk)
chunk_size = 5000  # Adjust as needed

try:
    with open(file_path, 'rb') as file:
        while True:
            try:
                chunk = pd.read_json(file, lines=True, nrows=chunk_size)
            except ValueError as e:
                print(f"Error reading JSON chunk: {e}")
                continue

            if chunk.empty:
                break

            # Process the chunk as needed

except FileNotFoundError:
    print(f"File not found: {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")



import json
import os

# Specify the path to the large JSON file
large_file_path = '../datasets/techmap-jobs_us_2023-05-05.json'

# Create a directory to store the smaller files
output_directory = 'techjobsuschunks'
os.makedirs(output_directory, exist_ok=True)

# Function to split the large JSON file into smaller JSON files
def split_large_json(input_path, chunk_size, output_dir):
    with open(input_path, 'r', encoding='utf-8') as input_file:  # Specify the encoding
        chunk_number = 0
        chunk_data = []

        for line in input_file:
            # Try to process each line as a separate JSON object
            try:
                json_obj = json.loads(line)
                chunk_data.append(json_obj)
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON line (line {chunk_number * chunk_size + len(chunk_data) + 1}): {e}")
                continue

            # If the chunk size is reached, save the chunk to a file
            if len(chunk_data) >= chunk_size:
                chunk_filename = os.path.join(output_dir, f'chunk_{chunk_number}.json')
                with open(chunk_filename, 'w', encoding='utf-8') as chunk_file:  # Specify the encoding
                    for obj in chunk_data:
                        json.dump(obj, chunk_file, ensure_ascii=False)  # Ensure non-ASCII characters are correctly encoded
                        chunk_file.write('\n')
                chunk_number += 1
                chunk_data = []

        # Save any remaining data as the last chunk
        if chunk_data:
            chunk_filename = os.path.join(output_dir, f'chunk_{chunk_number}.json')
            with open(chunk_filename, 'w', encoding='utf-8') as chunk_file:  # Specify the encoding
                for obj in chunk_data:
                    json.dump(obj, chunk_file, ensure_ascii=False)  # Ensure non-ASCII characters are correctly encoded
                    chunk_file.write('\n')

# Specify the desired chunk size (number of lines per chunk)
chunk_size = 5000  # Adjust as needed

# Call the function to split the large JSON file into smaller chunks
split_large_json(large_file_path, chunk_size, output_directory)


In [1]:
import json
import os

# Specify the path to the large JSON file
large_file_path = '../datasets/techmap-jobs_us_2023-05-05.json'

# Create a directory to store the smaller files
output_directory = 'techjobsuschunks'
os.makedirs(output_directory, exist_ok=True)

# Determine which chunks have already been processed
processed_chunks = set()
for filename in os.listdir(output_directory):
    if filename.startswith('chunk_') and filename.endswith('.json'):
        chunk_number = int(filename.split('_')[1].split('.')[0])
        processed_chunks.add(chunk_number)

# Find the next chunk number to process
next_chunk_number = max(processed_chunks) + 1 if processed_chunks else 0

# Function to split the large JSON file into smaller JSON files
def split_large_json(input_path, chunk_size, output_dir, start_chunk_number):
    with open(input_path, 'r', encoding='utf-8') as input_file:
        chunk_number = start_chunk_number
        chunk_data = []

        for line in input_file:
            # Try to process each line as a separate JSON object
            try:
                json_obj = json.loads(line)
                chunk_data.append(json_obj)
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON line (line {chunk_number * chunk_size + len(chunk_data) + 1}): {e}")
                continue

            # If the chunk size is reached, save the chunk to a file
            if len(chunk_data) >= chunk_size:
                chunk_filename = os.path.join(output_dir, f'chunk_{chunk_number}.json')
                with open(chunk_filename, 'w', encoding='utf-8') as chunk_file:
                    for obj in chunk_data:
                        json.dump(obj, chunk_file, ensure_ascii=False)
                        chunk_file.write('\n')
                chunk_number += 1
                chunk_data = []

        # Save any remaining data as the last chunk
        if chunk_data:
            chunk_filename = os.path.join(output_dir, f'chunk_{chunk_number}.json')
            with open(chunk_filename, 'w', encoding='utf-8') as chunk_file:
                for obj in chunk_data:
                    json.dump(obj, chunk_file, ensure_ascii=False)
                    chunk_file.write('\n')

# Specify the desired chunk size (number of lines per chunk)
chunk_size = 5000  # Adjust as needed

# Call the function to continue processing from the next chunk
split_large_json(large_file_path, chunk_size, output_directory, next_chunk_number)
